<a href="https://colab.research.google.com/github/dakilaledesma/herbarium_model_testing/blob/main/2x3xnet/H2022_GPU_DualNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%time
! unzip -q /content/drive/MyDrive/UNC/H2022/RCNN_PP/pls224p56_Orchidaceae_train.zip -d /content/
! unzip -q /content/drive/MyDrive/UNC/H2022/orchidaceae_train.zip -d /content/

CPU times: user 114 ms, sys: 18 ms, total: 132 ms
Wall time: 18.9 s


In [2]:
! pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.4 MB/s 


In [3]:
classes = 300

In [4]:
from glob import glob
import os
cat_to_int = {}
for i, f in enumerate(sorted(glob("pls224p56_Orchidaceae_train/*"))):
  cat = os.path.basename(f)
  cat_to_int[cat] = i

In [5]:
valid_fn = {}
for fn in sorted(glob("pls224p56_Orchidaceae_train/**/*.*", recursive=True)):
  bn = os.path.basename(fn)
  valid_fn[bn] = fn

In [6]:
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tqdm.notebook import tqdm
import numpy as np

o = []
pl = []
y = []
orc = sorted(glob("orchidaceae_train/**/*.*", recursive=True))
for fn in tqdm(orc, total=len(list(orc))):
  bn =  os.path.basename(fn)
  cat = cat_to_int[fn.split("/")[-2]]
  try:
    pl_fn = valid_fn[bn]
  except KeyError:
    pass

  img = image.load_img(fn, target_size=(224, 224))
  x = image.img_to_array(img)
  x = preprocess_input(x)
  o.append(x)

  img = image.load_img(pl_fn, target_size=(224, 224))
  x = image.img_to_array(img)
  x = preprocess_input(x)
  pl.append(x)

  one_hot = np.zeros((classes,), dtype='int')
  one_hot[cat] = 1
  y.append(one_hot)


  0%|          | 0/9419 [00:00<?, ?it/s]

In [9]:
import tensorflow_addons as tfa
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Concatenate, Flatten, GlobalAveragePooling2D
import numpy as np

classes = 300

orig_model = ResNet50V2(weights='imagenet', include_top=False)
lg_model = ResNet50V2(weights='imagenet', include_top=False)

for ln in [layer.name for layer in lg_model.layers]:
    lg_model.get_layer(name=ln)._name = f"lg_{ln}"

out = Concatenate()([orig_model.output, lg_model.output])
out = GlobalAveragePooling2D()(out)
out = Dense(512, activation='relu')(out)
out = Dense(classes, activation='softmax', name='predictions')(out)

model = Model(inputs=[orig_model.input, lg_model.input], outputs=out)
opt = tfa.optimizers.AdaBelief(learning_rate=1e-3)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics='accuracy')

94683136/94668760 [==============================] - 1s 0us/step


/usr/local/lib/python3.7/dist-packages/tensorflow_addons/optimizers/adabelief.py:129: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [10]:
o = np.array(o)
pl = np.array(pl)

In [11]:
y = np.array(y)

In [12]:
model.fit(x=[o, pl], y=y, batch_size=16, epochs=24)

Epoch 1/24
589/589 [==============================] - 211s 245ms/step - loss: 4.4232 - accuracy: 0.1302
Epoch 2/24
589/589 [==============================] - 144s 245ms/step - loss: 2.9230 - accuracy: 0.3101
Epoch 3/24
589/589 [==============================] - 144s 245ms/step - loss: 2.1557 - accuracy: 0.4553
Epoch 4/24
589/589 [==============================] - 144s 244ms/step - loss: 1.6394 - accuracy: 0.5614
Epoch 5/24
589/589 [==============================] - 144s 244ms/step - loss: 1.2771 - accuracy: 0.6428
Epoch 6/24
589/589 [==============================] - 144s 244ms/step - loss: 0.9819 - accuracy: 0.7210
Epoch 7/24
589/589 [==============================] - 143s 243ms/step - loss: 0.7867 - accuracy: 0.7718
Epoch 8/24
589/589 [==============================] - 144s 244ms/step - loss: 0.6029 - accuracy: 0.8202
Epoch 9/24
589/589 [==============================] - 143s 243ms/step - loss: 0.5227 - accuracy: 0.8449
Epoch 10/24
589/589 [==============================] - 143s 243m

In [13]:
model.save("/content/drive/MyDrive/UNC/H2022/DualNet2_adabelief")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/UNC/H2022/DualNet2_adabelief/assets
